In [285]:
companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',  'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
         'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK', 'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS','MAGN', 
         'PHOR', 'RTKM', 'VTBR', 'TATNP', 'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS','FEES', 'TRNFP',
         'CBOM', 'VKCO', 'ENPG', 'SGZH', 'FIVE',  'MOEX', 'GLTR', 'HYDR', 'FIXP']

In [286]:
import pandas as pd
import numpy as np
import cvxpy as cvx

In [242]:
def expected_returns(data, day, hour, companies):
    output = np.zeros(len(companies))
    preds = data.loc[day, hour]
    availible_companies = preds.Asset.values
    
    for i, company in enumerate(companies):
        if company in availible_companies:
            output[i] = preds[preds.Asset == company].pct_pred[0]
        else:
            output[i] = 0
    return output

def real_returns(data, day, hour, companies):
    output = np.zeros(len(companies))
    preds = data.loc[day, hour]
    availible_companies = preds.Asset.values
    
    for i, company in enumerate(companies):
        if company in availible_companies:
            output[i] = preds[preds.Asset == company].pct_fact[0]
        else:
            output[i] = 0
    return output 

In [193]:
data = pd.read_csv('backtest_data').rename(columns={'Unnamed: 0':'Date'})

data = data[data.Asset != 'POLY']
bad_days = list( data.groupby('date').aggregate({'Asset':'count'}).query('Asset < 200').index )


for i in range(len(bad_days)):
    data = data[data.Date != bad_days[i]]
    
data.index = [data.date, data.hour]    

In [197]:
trading_days = sorted(data.date.unique()) #Набор торговых дней с предсказаниями и фактами
trading_hours = data.hour.unique()

In [245]:
day

'2023-10-24'

In [246]:
hour

1

In [252]:
pred_ret = expected_returns(data, day, hour, companies)

/tmp/ipykernel_24902/179374125.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  preds = data.loc[day, hour]


In [260]:
n = len(companies)

In [261]:
x = cvx.Variable(n)
t_cost = np.zeros(len(companies)) + 0.003

In [262]:
objective = cvx.Maximize( pred_ret@x - t_cost@x )

In [282]:
constraints = [cvx.sum(x) + t_cost@x == 0, cvx.sum(x)==1000] 

In [283]:
prob = cvx.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(solver='SCIPY',verbose=True)
# The optimal value for x is stored in `x.value`.
print(x.value)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Feb 23 04:35:03 PM: Your problem has 41 variables, 2 constraints, and 0 parameters.
(CVXPY) Feb 23 04:35:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 23 04:35:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 23 04:35:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 23 04:35:03 PM: Compiling problem (target solver=SCIPY).
(CVXPY) Feb 23 04:35:03 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

In [284]:
# The optimal value for x is stored in `x.value`.
print(x.value)

None


In [ ]:
# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)